# Abstract:
- This dataset is intended to represent the potable ground-water resource.
- This dataset therefore does not include thermal and saline water *(temperature greater than `50℃` or dissolved solids greater than `3000 ㎎/L` or specific conductance greater than `4000 µS/㎝`)*.
- In addition, this dataset includes only the most recent arsenic analysis available for each well, and only analyses performed by hydride generation or ICP/MS.

In [1]:
import pandas as pd
import datetime
from state_names import name_this

# Basic documentation of dataset elements:
- `STAID`: USGS station identifier, based on latitude and longitude *(each well in the National Water Information System has an unique USGS station identifier)*
- `STATE`: Two-letter postal code for the U.S. state in which the sample was collected.
- `FIPS`: Federal Information Processing Standard state and county codes (see http://www.itl.nist.gov/fipspubs/fip6-4.htm )
- `LAT_DMS`: Latitude of well, *in degrees, minutes, and seconds*
- `LONG_DMS`: Longitude of well, *in degrees, minutes, and seconds*
- `WELLDPTH`: Depth *(below land surface)* of finished well *(may be less than total hole depth drilled)* ***(feet)***
- `SAMPDATE`: Date water sample was collected
- `SAMPTIME`: Time water sample was collected
- `AS_RMRK`: Remark code qualifying the analytical result in AS_CONC.
    - For example, the combination of AS_RMRK="<" and AS_CONC="1" indicates that arsenic was not detectable *at a laboratory reporting limit of 1 microgram (㎍) per liter.*
- `AS_CONC`: Concentration of arsenic in sample, in micrograms per liter (㎍/L) as arsenic
- `LAT_DD`: Latitude of well, *in decimal degrees*
- `LON_DD`: Longitude of well, *in decimal degrees*

In [2]:
# Column 1 needs to be passed in as a list in "skiprows", otherwise the header is skipped too.
df = pd.read_csv('Resources/arsenic_nov2001.csv', skiprows = [1]) #, index_col='STAID'

# Many columns *frustratingly* have spaces in their names . . . This fixes that issue.
df = df.rename(columns={'LAT_DMS ': 'LAT_DMS',
                        ' LON_DMS': 'LON_DMS',
                        'WELLDPTH  ':'WELLDPTH',
                        'SAMPDATE ':'SAMPDATE'})
df = df.fillna(value=0)
df.head()

,STAID,STATE,FIPS,LAT_DMS,LON_DMS,WELLDPTH,SAMPDATE,SAMPTIME,AS_RMRK,AS_CONC,LAT_DD,LON_DD
0,182113064451900,VI,78020,182113,644519,100.0,31-Aug-92,1020,<,1.0,18.35361,64.75528
1,182109064460300,VI,78020,182109,644603,60.0,26-Aug-92,1130,<,1.0,18.35250,64.76750
2,180859065474100,PR,72069,180859,654741,0.0,07-Jul-82,940,<,1.0,18.14972,65.79472
3,180339065523300,PR,72151,180339,655233,120.0,07-Jul-82,1645,<,1.0,18.06083,65.87583
4,175836066021400,PR,72015,175836,660214,0.0,12-Aug-86,1545,<,1.0,17.97667,66.03722


In [3]:
df['SAMPDATE'] = pd.to_datetime(df['SAMPDATE'])

In [4]:
slim_df = df[['STAID', 'SAMPDATE', 'STATE', 'LAT_DD', 'LON_DD', 'WELLDPTH', 'AS_CONC']]
slim_df = slim_df.rename(columns={'LAT_DD': 'LAT', 'LON_DD': 'LON', 'WELLDPTH': 'WELL_DEPTH'})
slim_df.head()

,STAID,SAMPDATE,STATE,LAT,LON,WELL_DEPTH,AS_CONC
0,182113064451900,1992-08-31,VI,18.35361,64.75528,100.0,1.0
1,182109064460300,1992-08-26,VI,18.35250,64.76750,60.0,1.0
2,180859065474100,1982-07-07,PR,18.14972,65.79472,0.0,1.0
3,180339065523300,1982-07-07,PR,18.06083,65.87583,120.0,1.0
4,175836066021400,1986-08-12,PR,17.97667,66.03722,0.0,1.0


In [5]:
slim_df.dtypes

STAID                  int64
SAMPDATE      datetime64[ns]
STATE                 object
LAT                  float64
LON                  float64
WELL_DEPTH           float64
AS_CONC              float64
dtype: object

In [6]:
for i in range(len(slim_df)):
    abbreviation = slim_df.iloc[i][2]
    state_name = name_this(abbreviation)
    slim_df.iat[i,2] = state_name

In [7]:
values = slim_df['STATE'].value_counts()
values

Idaho             2194
California        2180
New Jersey        1428
Washington        1046
Arizona            911
Colorado           897
New Mexico         731
Pennsylvania       600
Nevada             583
Florida            581
Texas              570
Michigan           554
Tennessee          455
Utah               443
Maryland           426
South Dakota       402
Nebraska           400
Missouri           348
Oklahoma           326
Louisiana          299
Indiana            287
Ohio               273
Wisconsin          260
North Dakota       258
New Hampshire      255
Oregon             249
Wyoming            236
Kansas             230
West Virginia      224
Arkansas           216
Illinois           189
Connecticut        188
Montana            181
Massachusetts      179
Alaska             167
Alabama            159
North Carolina     148
Iowa               130
Minnesota          125
Delaware           120
Virginia           111
Georgia            109
Puerto Rico        103
South Carol

In [8]:
max_lon = -103.771556
min_lon = max_lon
max_lat = 44.967243
min_lat = max_lat
for i in range(len(slim_df)):
    lat = slim_df.iloc[i][3]
    lon = slim_df.iloc[i][4]
    # This IF statement here corrects any longitudes incorrectly input as a positive value.
    if (lon > 0):
        # Since the Aleutian Islands of Alaskan DO cross over the 180ᵗʰ Meridian, this check is added.
        if (lat > 51) and (lat < 56):
            # The Aleutian Islands DO NOT cross over the 165ᵗʰ Meridian East, thus 165 can be used as a cap.
            if (lon < 165):
                lon *= -1
                slim_df.iat[i, 4] = lon
        # This includes Guam and the The Northern Mariana Islands, which form a narrow island chain.
        if (lat > 13) and (lat < 21):
            # Because the longitude is kept narrow, we avoid confusion overlapping with Hawaii.
            if (lon > 146.5) or (lon < 144.5):
                lon *= -1
                slim_df.iat[i, 4] = lon
        else:
            lon *= -1
            slim_df.iat[i, 4] = lon
        # As a result, we maintain the integrity of our coordinates.
        # All regions in the US which are East of the Prime Meridian are accounted for, 
        # and everything else appearing East has its longitude flipped and corrected.
        
    if (lat > max_lat):
        max_lat = lat
    elif (lat < min_lat):
        min_lat = lat
    if (lon > max_lon):
        max_lon = lon
    elif (lon < min_lon):
        min_lon = lon

In [9]:
print(f'Between\t {min_lat},{min_lon}')
print(f'And\t {max_lat},{max_lon}')

Between	 17.95889,-158.63194
And	 68.14333,-64.75528


In [10]:
mid_lat = ((max_lat - min_lat)/2) + min_lat
mid_lon = ((max_lon - min_lon)/2) + min_lon
print(f'{mid_lat},{mid_lon}')

43.05111000000001,-111.69360999999999


In [11]:
slim_df.head()

,STAID,SAMPDATE,STATE,LAT,LON,WELL_DEPTH,AS_CONC
0,182113064451900,1992-08-31,Virgin Islands,18.35361,-64.75528,100.0,1.0
1,182109064460300,1992-08-26,Virgin Islands,18.35250,-64.76750,60.0,1.0
2,180859065474100,1982-07-07,Puerto Rico,18.14972,-65.79472,0.0,1.0
3,180339065523300,1982-07-07,Puerto Rico,18.06083,-65.87583,120.0,1.0
4,175836066021400,1986-08-12,Puerto Rico,17.97667,-66.03722,0.0,1.0


In [12]:
slim_df.to_csv('Resources/work_in_progress.csv', sep=',', header=True)

In [12]:
features_and_target_df = slim_df[['LAT', 'LON', 'WELL_DEPTH', 'AS_CONC']]
features_and_target_df.head()

,LAT,LON,WELL_DEPTH,AS_CONC
0,18.35361,-64.75528,100.0,1.0
1,18.35250,-64.76750,60.0,1.0
2,18.14972,-65.79472,0.0,1.0
3,18.06083,-65.87583,120.0,1.0
4,17.97667,-66.03722,0.0,1.0


In [13]:
features_and_target_df.to_csv('Resources/features_and_target.csv', sep=',', header=True, index=False)

In [ ]:
california = []
for i in range(len(slim_df)):
    sampdate = slim_df.iloc[i][1]
    state = slim_df.iloc[i][2]
    lat = slim_df.iloc[i][3]
    lon = slim_df.iloc[i][4]
    as_conc = slim_df.iloc[i][5]
    if state == "California":
        add_entry = {
            'SAMPDATE': sampdate,
            'LAT': lat,
            'LON': lon,
            'AS_CONC': as_conc
        }
        california.append(add_entry)
california

In [ ]:
nearby = []
for i in range(len(slim_df)):
    sampdate = slim_df.iloc[i][1]
    state = slim_df.iloc[i][2]
    lat = slim_df.iloc[i][3]
    lon = slim_df.iloc[i][4]
    as_conc = slim_df.iloc[i][5]
    if (lat > 38.75) and (lat < 38.85):
        if (lon > -121.3) and (lon < -121.2):
            add_entry = {
                'SAMPDATE': sampdate,
                'LAT': lat,
                'LON': lon,
                'AS_CONC': as_conc
            }
            nearby.append(add_entry)
nearby

In [ ]:
cal_32 = []
cal_33 = []
cal_34 = []
cal_35 = []
cal_36 = []
cal_37 = []
cal_38 = []
cal_39 = []
cal_40 = []
for entry in california:
    if (entry['LAT'] > 32) and (entry['LAT'] < 33):
        cal_32.append(entry)
    elif (entry['LAT'] < 34):
        cal_33.append(entry)
    elif (entry['LAT'] < 35):
        cal_34.append(entry)
    elif (entry['LAT'] < 36):
        cal_35.append(entry)
    elif (entry['LAT'] < 37):
        cal_36.append(entry)
    elif (entry['LAT'] < 38):
        cal_37.append(entry)
    elif (entry['LAT'] < 39):
        cal_38.append(entry)
    elif (entry['LAT'] < 40):
        cal_39.append(entry)
    elif (entry['LAT'] < 41):
        cal_40.append(entry)
print(f'32° - 33°: {len(cal_32)}')
print(f'33° - 34°: {len(cal_33)}')
print(f'34° - 35°: {len(cal_34)}')
print(f'35° - 36°: {len(cal_35)}')
print(f'36° - 37°: {len(cal_36)}')
print(f'37° - 38°: {len(cal_37)}')
print(f'38° - 39°: {len(cal_38)}')
print(f'39° - 40°: {len(cal_39)}')
print(f'40° - 41°: {len(cal_40)}')